# Do post-impressionist painters paint with brighter colors than their predecessors? 

Hypothesis: yes. But by how much, precisely? 

Import some necessary libraries: 

In [154]:
import pandas as pd
from urllib.request import urlretrieve 
from time import sleep
import json
import colorgram
from urllib.parse import quote
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

Load data gathered from Wikidata. 

Get post-impressionist paintings using [this query](https://query.wikidata.org/#%23defaultView%3AImageGrid%0ASELECT%20%3Fimg%20%0AWHERE%0A%7B%0A%20%20%3Fpainting%20wdt%3AP31%20wd%3AQ3305213.%0A%20%20%3Fpainting%20wdt%3AP135%20wd%3AQ166713.%0A%20%20%3Fpainting%20wdt%3AP18%20%3Fimg%20.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%7D%0A%0A) in Wikidata: 

```sparql
#defaultView:ImageGrid
SELECT ?img 
WHERE
{
  ?painting wdt:P31 wd:Q3305213.
  ?painting wdt:P135 wd:Q166713.
  ?painting wdt:P18 ?img .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} LIMIT 500
```

And get early 19th Century paintings using [this query](https://query.wikidata.org/#%23defaultView%3AImageGrid%0ASELECT%20%3Fimg%20%3Fdate%0AWHERE%0A%7B%0A%20%20%3Fpainting%20wdt%3AP31%20wd%3AQ3305213.%0A%20%20%3Fpainting%20wdt%3AP571%20%3Fdate.%0A%20%20%3Fpainting%20wdt%3AP18%20%3Fimg%20.%0A%20%20FILTER%28YEAR%28%3Fdate%29%20%3E%201800%20%26%26%20YEAR%28%3Fdate%29%20%3C%201850%29.%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%7D%20LIMIT%20500): 

```sparql
#defaultView:ImageGrid
SELECT ?img ?date
WHERE
{
  ?painting wdt:P31 wd:Q3305213.
  ?painting wdt:P571 ?date.
  ?painting wdt:P18 ?img .
  FILTER(YEAR(?date) > 1800 && YEAR(?date) < 1850).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} LIMIT 500
```

Load the resulting files: 

In [155]:
with open('early19thC.json') as f: 
    early = json.load(f)

In [156]:
with open('postimpressionism.json') as f:
    late = json.load(f)

Truncate to the first 100, so that we're comparing equal amounts in each: 

In [157]:
urls = {"early": [item['img'] for item in early][:100], 
        "late": [item['img'] for item in late][:100]}

Download thumbnail images of them locally: 

In [158]:
def getThumbnailUrl(imageName, width=100):
    """Return the URL to the thumbnail of the file, at the given width."""
    imageName = imageName.split('/')[-1]
    return f"http://commons.wikimedia.org/w/index.php?title=Special:FilePath&file={imageName}&width={width}"

In [160]:
# Only run this once
def downloadImages():
    for label, urlList in urls.items(): 
        for i in range(len(urlList)): 
            url = getThumbnailUrl(urlList[i])
            #print(f"Getting {url}")
            dest = f"img/{label}/{i:03d}.jpg"
            #print(f"saving {url} to {dest}")
            urlretrieve(url, dest)
            sleep(1)

Use [colorgram](https://github.com/obskyr/colorgram.py) to extract dominant colors from the images: 

In [161]:
def getDominantColor(img): 
    color = colorgram.extract(img, 1)[0]
    return color.hsl

In [162]:
colors = {label: [getDominantColor(img) for img in glob(f'img/{label}/*')] for label in urls} 

Convert to HSL (hue, saturation, luminosity): 

In [163]:
saturation = {label: [c.s for c in colors[label]] for label in colors}
luminosity = {label: [c.l for c in colors[label]] for label in colors}

Calculate averages for each category: 

In [164]:
meanSats = {label: np.mean(np.array(sats)) for label, sats in saturation.items()}

In [165]:
meanLums = {label: np.mean(np.array(sats)) for label, sats in luminosity.items()}

Now calculate how much larger post-impressionist luminosity is, on average: 

In [167]:
luminosityChangePercent = (meanLums['late'] - meanLums['early']) / meanLums['early'] * 100

In [173]:
print(f"Post-impressionist painters are {luminosityChangePercent:.1f}% more luminous than painters in the early 19th century!")

Post-impressionist painters are 37.4% more luminous than painters in the early 19th century!
